In [1]:
!pip install transformers

  Using cached transformers-3.0.2-py3-none-any.whl (769 kB)
  Using cached sentencepiece-0.1.91-cp37-cp37m-win_amd64.whl (1.2 MB)
Processing c:\users\shamazha3\appdata\local\pip\cache\wheels\69\09\d1\bf058f7d6fa0ecba2ce7c66be3b8d012beb4bf61a6e0c101c0\sacremoses-0.0.43-py3-none-any.whl
  Using cached requests-2.24.0-py2.py3-none-any.whl (61 kB)
  Using cached tokenizers-0.8.1rc1-cp37-cp37m-win_amd64.whl (1.9 MB)
  Using cached regex-2020.7.14-cp37-cp37m-win_amd64.whl (268 kB)
  Using cached filelock-3.0.12-py3-none-any.whl (7.6 kB)
  Using cached click-7.1.2-py2.py3-none-any.whl (82 kB)
  Using cached joblib-0.16.0-py3-none-any.whl (300 kB)
  Using cached urllib3-1.25.9-py2.py3-none-any.whl (126 kB)
  Using cached chardet-3.0.4-py2.py3-none-any.whl (133 kB)
  Using cached idna-2.10-py2.py3-none-any.whl (58 kB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.8.0
    Uninstalling tokenizers-0.8.0:
      Successfully uninstalled tokenizers-0.8.0


In [23]:
pip install --upgrade tensorflow

Requirement already up-to-date: tensorflow in c:\users\shamazha3\anaconda3\envs\nlp\lib\site-packages (2.2.0)
Note: you may need to restart the kernel to use updated packages.


In [14]:
import transformers
transformers.__version__

'3.0.2'

In [6]:
pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to c:\users\shamaz~1\appdata\local\temp\pip-req-build-pzgl3dv3
  Created wheel for transformers: filename=transformers-3.0.2-py3-none-any.whl size=798088 sha256=bda735121163be5c6c143d209eb63ada0a607acc1907eb9867a493b3f388ef2d
  Stored in directory: C:\Users\SHAMAZ~1\AppData\Local\Temp\pip-ephem-wheel-cache-7rub2a2p\wheels\35\2e\a7\d819e3310040329f0f47e57c9e3e7a7338aa5e74c49acfe522
Successfully built transformers
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.8.1rc1
    Uninstalling tokenizers-0.8.1rc1:
      Successfully uninstalled tokenizers-0.8.1rc1



  Running command git clone -q https://github.com/huggingface/transformers 'C:\Users\SHAMAZ~1\AppData\Local\Temp\pip-req-build-pzgl3dv3'
ERROR: transformers 3.0.2 has requirement tokenizers==0.8.1.rc1, but you'll have tokenizers 0.8.1rc2 which is incompatible.
ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'c:\\users\\shamazha3\\anaconda3\\envs\\txt\\lib\\site-packages\\~okenizers\\tokenizers.cp37-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [13]:
import numpy as np
import pandas as pd
import os
from transformers import BertTokenizer
import tokenizer
import string
import torch
import transformers
import torch.nn as nn
from tqdm import tqdm
import re


import torch.nn.functional as F
from torch.optim import lr_scheduler

from sklearn import model_selection
from sklearn import metrics
import transformers
import tokenizers
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
from tqdm.autonotebook import tqdm
import utils

In [15]:
class AverageMeter:
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count



def jaccard(str1,str2):
    a=set(str1.lower().split())
    b=set(str2.lower().split())
    c=a.intersection(b)
    return float(len(c))/(len(a)+len(b)-len(c))

In [16]:
train=pd.read_csv(r'C:\Users\shamazha3\Desktop\Text\tweet-sentiment-extraction\train.csv')
test=pd.read_csv(r'C:\Users\shamazha3\Desktop\Text\tweet-sentiment-extraction\test.csv')

In [17]:
MAX_LEN = 128
TRAIN_BATCH_SIZE = 64
VALID_BATCH_SIZE = 16
EPOCHS = 5
BERT_PATH = r"C:\Users\shamazha3\Desktop\Text\Bert files" 
MODEL_PATH = r"C:\Users\shamazha3\Desktop\Text\Bert files\bert-base-uncased-pytorch_model.bin"
TRAINING_FILE = train
TOKENIZER = tokenizers.BertWordPieceTokenizer(
    os.path.join(BERT_PATH,"vocab.txt"), 
    lowercase=True
    )
    

## Dataset

In [18]:
class TweetDataset:
    def __init__(self, tweet, sentiment, selected_text):
        self.tweet = tweet
        self.sentiment = sentiment
        self.selected_text = selected_text
        self.tokenizer = TOKENIZER
        self.max_len = MAX_LEN
    
    def __len__(self): #length fo the dataset
        return len(self.tweet)
    
    def __getitem__(self, item):  #get training data and preprocessing
        tweet = " ".join(str(self.tweet[item]).split())
        selected_text = " ".join(str(self.selected_text[item]).split())
        
        len_st = len(selected_text)
        idx0 = -1
        idx1 = -1
        for ind in (i for i, e in enumerate(tweet) if e == selected_text[0]): #match the index of characters in text and selected text
            if tweet[ind: ind+len_st] == selected_text:
                idx0 = ind  #start ind
                idx1 = ind + len_st - 1  #end indz
                break
        #find character indeces
        char_targets = [0] * len(tweet)   #Create a vector with zero values
        #[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
        if idx0 != -1 and idx1 != -1:
            for j in range(idx0, idx1 + 1):
                if tweet[j] != " ":
                    char_targets[j] = 1
        #[0,0,0,0,1,1,1,0,1,1,1,1,0,0,0]
        
        tok_tweet = self.tokenizer.encode(sequence=self.sentiment[item], pair=tweet)
        tok_tweet_tokens = tok_tweet.tokens
        tok_tweet_ids = tok_tweet.ids
        tok_tweet_offsets = tok_tweet.offsets[3:-1]
        # print(tok_tweet_tokens)
        # print(tok_tweet.offsets)
        # ['[CLS]', 'spent', 'the', 'entire', 'morning', 'in', 'a', 'meeting', 'w', '/', 
        # 'a', 'vendor', ',', 'and', 'my', 'boss', 'was', 'not', 'happy', 'w', '/', 'them', 
        # '.', 'lots', 'of', 'fun', '.', 'i', 'had', 'other', 'plans', 'for', 'my', 'morning', '[SEP]']
        
        #Creating target variable
        targets = [0] * (len(tok_tweet_tokens) - 4)
        if self.sentiment[item] == "positive" or self.sentiment[item] == "negative":
            sub_minus = 8
        else:
            sub_minus = 7
        
        #[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]        
        for j, (offset1, offset2) in enumerate(tok_tweet_offsets):
            if sum(char_targets[offset1 - sub_minus:offset2 - sub_minus]) > 0: #checking for a partial match to label as 1
                targets[j] = 1
        #[0,0,0,0,1,1,1,0,1,1,1,1,0,0,0]        
        
        targets = [0] + [0] + [0] + targets + [0]  #cls, target and sep
        targets_start = [0] * len(targets)
        targets_end = [0] * len(targets)

        non_zero = np.nonzero(targets)[0]
        if len(non_zero) > 0:
            targets_start[non_zero[0]] = 1
            targets_end[non_zero[-1]] = 1
        
        #print(targets_start)
        #print(targets_end)

        # Mask
        mask = [1] * len(tok_tweet_ids)
        token_type_ids = [0] * 3 + [1] * (len(tok_tweet_ids) - 3)

        #Padding
        padding_length = self.max_len - len(tok_tweet_ids)
        ids = tok_tweet_ids + ([0] * padding_length)
        mask = mask + ([0] * padding_length)
        token_type_ids = token_type_ids + ([0] * padding_length)
        targets = targets + ([0] * padding_length)
        targets_start = targets_start + ([0] * padding_length)
        targets_end = targets_end + ([0] * padding_length)

        sentiment = [1, 0, 0]
        if self.sentiment[item] == "positive":
            sentiment = [0, 0, 1]
        if self.sentiment[item] == "negative":
            sentiment = [0, 1, 0]

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'tweet_tokens': " ".join(tok_tweet_tokens),
            'targets': torch.tensor(targets, dtype=torch.long),
            'targets_start': torch.tensor(targets_start, dtype=torch.long),
            'targets_end': torch.tensor(targets_end, dtype=torch.long),
            'padding_len': torch.tensor(padding_length, dtype=torch.long),
            'orig_tweet': self.tweet[item],
            'orig_selected': self.selected_text[item],
            'sentiment': torch.tensor(sentiment, dtype=torch.float),
            'orig_sentiment': self.sentiment[item]
        }
    

dset = TweetDataset( tweet=train.text.values, sentiment=train.sentiment.values, selected_text=train.selected_text.values )
print(dset[0])



{'ids': tensor([ 101, 8699,  102, 1045, 1036, 1040, 2031, 5838, 1010, 2065, 1045, 2020,
        2183,  102,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]), 'mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

## Model

In [57]:
class BERTBaseUncased(nn.Module):
    def __init__(self):
        super(BERTBaseUncased, self).__init__()
        self.bert = transformers.BertModel.from_pretrained(r'C:\Users\shamazha3\Desktop\Text\Bert files\bert_config.json')
        self.l0 = nn.Linear(768, 2) # Linear layer
#r'C:\Users\shamazha3\Desktop\Text\Bert files\model.bin'   
#r'C:\Users\shamazha3\Desktop\Text\Bert files\bert_config.json'
    def forward(self, ids, mask, token_type_ids, sentiment):
        
        
        o1, o2 = self.bert(
            ids, 
            attention_mask=mask,
            token_type_ids=token_type_ids
        )
        output= self.bert(
            ids, 
            attention_mask=mask,
            token_type_ids=token_type_ids
        )
        sequence_output = output[0]
        pooled_output = output[1]
        
        # (batch_size, num_tokens, 768) #sequence output
        logits = self.l0(sequence_output)
        # (batch_size, num_tokens, 2) split into two 
        # (batch_size, num_tokens, 1), (batch_size, num_tokens, 1)
        start_logits, end_logits = logits.split(1, dim=-1) 
        start_logits = start_logits.squeeze(-1)  #where selected text is beginning
        end_logits = end_logits.squeeze(-1)  #where selected text is ending
        # (batch_size, num_tokens), (batch_size, num_tokens)
        
        start_logits = self.out1(torch.cat(start_logits, sentiment) , 1)
        start_logits = self.out2(torch.cat(end_logits, sentiment) , 1)

        return start_logits, end_logits

In [56]:
def loss_fn(o1,o2,t1,t2):              #output1,2 & target 1,2
    l1=nn.BCEWithLogitsLoss()(o1,t1)
    l2=nn.BCEWithLogitsLoss()(o2,t2)
    return l1+l2

def train_fun(data_loader, model, optimizer, device, scheduler):
    model.train()
    losses =AverageMeter()
    tk0=tqdm(data_loader, total=len(data_loader))
    with torch.no_grad():
        
        for bi, d in tqdm(enumerate(tk0), total=len(data_loader)):
            ids = d["ids"]
            token_type_ids = d["token_type_ids"]
            mask = d["mask"]
            tweet_tokens = d["tweet_tokens"]
            target_start = d["target_start"]
            target_end = d["target_end"]
            
            target_start = target_start.to(device, dtype=torch.float)
            target_end = target_end.to(device, dtype=torch.float)            
            ids = ids.to(device, dtype=torch.long)
            token_type_ids = token_type_ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            sentiment = sentiment.to(device, dtype=torch.float)

            optimizer.zero_grad()
            o1, o2 = model(
                ids=ids,
                mask=mask,
                token_type_ids=token_type_ids
            )
            loss=loss_fn(o1,o2, targets_start, targets_end)
            loss.backward()
            optimizer.step()
            scheduler.step()
            losses.update(loss.item(),ids.size(0))
            tk0.set_postfix(loss=losses.avg)
            
final_output = ""            
def eval_fun(data_loader, model, device):
    model.eval()
    fin_output_start = []
    fin_output_end = []
    fin_padding_lens = []
    fin_tweet_tokens = []
    fin_orig_sentiment = []
    fin_orig_selected = []
    fin_orig_tweet = []
    
    with torch.no_grad():
        
        for bi, d in tqdm(enumerate(data_loader), total=len(data_loader)):
            ids = d["ids"]
            token_type_ids = d["token_type_ids"]
            mask = d["mask"]
            tweet_tokens = d["tweet_tokens"]
            padding_len = d["padding_len"]
            sentiment = d["sentiment"]
            orig_selected = d["orig_selected"]
            orig_sentiment = d["orig_sentiment"]
            orig_tweet = d["orig_tweet"]
            
            
                       
            ids = ids.to(device, dtype=torch.long)
            token_type_ids = token_type_ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            sentiment = sentiment.to(device, dtype=torch.float)

            
            o1, o2 = model(
                ids=ids,
                mask=mask,
                token_type_ids=token_type_ids
            )
            
            fin_output_start.append(torch.sigmoid(o1).cpu().detach().numpy())
            fin_output_end.append(torch.sigmoid(o1).cpu().detach().numpy())
            fin_padding_lens.extend(padding_len.cpu().detach().numpy())
            
            fin_tweet_tokens.extend(tweet_tokens)
            fin_orig_sentiment.extend(orig_sentiment)
            fin_orig_selected.extend(orig_selected)
            fin_orig_tweet.extend(orig_tweet)
            
        fin_output_start = np.vstack(fin_output_start)
        fin_output_end = np.vstack(fin_output_end) #create a numpy array
        
        #we will store some values
        threshhold = 0.2        #>0.2 is our candidate
        jaccards = []
        for j in range(len(fin_tweet_tokens)): #loop in prdiction
            target_string = fin_orig_selected[j]
            tweet_tokens = fin_tweet_tokens[j]
            padding_len = fin_padding_len[j]
            original_tweet = fin_orig_tweet[j]
            sentiment = fin_orig_sentiment[j]
            
            if padding_len > 0:
                mask_start = fin_output_start[j, :][:-padding_len] >= threshhold
                mask_end = fin_output_end[j, :][:-padding_len] >= threshhold
            else:    
                mask_start = fin_output_start[j, :] >= threshhold
                mask_end = fin_output_end[j, :] >= threshhold
                
            mask = [0] * len(mask_start)
            idx_start = np.nonzero(mask_start)[0]
            idx_end = np.nonzero(mask_end)[0]
            
            if len(idx_start) > 0:
                idx_start = idx_start[0]
                if len(idx_end) > 0:
                    idx_end = idx_end[0]
                else:
                    idx_start = 0
                    idx_end = 0
                    
            for mj in range(idx_Start, idx_end + 1):
                mask[mj] = 1
                
            output_tokens = [x for p, x in enumerate(tweet_tokens.split()) if mask[p] == 1]
            output_tokens = [x for x in output_tokens if x not in ("[CLS]", "[SEP]")]
            
#             final_output = ""
            for ot in output_tokens:
                if ot.startswith("##"):
                    final_output = final_output + ot[2:]
                elif len(ot) ==1 and ot in string.punctuation:
                    final_output = final_output + ot
                else:
                     final_output = final_output + " " + ot
            final_output = final_output.strip()
            print(final_output)
            # if sentiment is neutral
            if sentiment  == "neutral" or len(original_tweet.split()) < 4:
                final_output = original_tweet
            
            jac = utils.jaccard(target_string.strip(), final_output.strip())
            jaccards.append(jac)
            
        mean_jac = np.mean(jaccards)
    return mean_jac
            
      

## Train

In [53]:
def mod():
    dfx= pd.read_csv(r'C:\Users\shamazha3\Desktop\Text\tweet-sentiment-extraction\train.csv', nrows=1000).dropna().reset_index(drop=True)
    
 
    df_train, df_valid = model_selection.train_test_split(
        dfx, test_size=0.1, random_state=42, stratify=dfx.sentiment.values
    )

    df_train = df_train.reset_index(drop=True)
    df_valid = df_valid.reset_index(drop=True)

    train_dataset = TweetDataset(
        tweet=df_train.text.values, sentiment=df_train.sentiment.values,selected_text=df_train.selected_text.values
    )

    train_data_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=TRAIN_BATCH_SIZE, num_workers=4
    )

    valid_dataset = TweetDataset(
        tweet=df_valid.text.values, sentiment=df_valid.sentiment.values,selected_text=df_valid.selected_text.values
    )

    valid_data_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=VALID_BATCH_SIZE, num_workers=1
    )

    device = torch.device("cpu")
    model = BERTBaseUncased()
    model.to(device)

    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [
        {
            "params": [
                p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
            ],
            "weight_decay": 0.001,
        },
        {
            "params": [
                p for n, p in param_optimizer if any(nd in n for nd in no_decay)
            ],
            "weight_decay": 0.0,
        },
    ]

    num_train_steps = int(len(df_train) / TRAIN_BATCH_SIZE * EPOCHS)
    optimizer = AdamW(optimizer_parameters, lr=3e-5)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=0, num_training_steps=num_train_steps
    )

#     model = nn.DataParallel(model)

    best_jaccard = 0
    for epoch in range(EPOCHS):
        engine.train_fn(train_data_loader, model, optimizer, device, scheduler)
        jaccard = engine.eval_fn(valid_data_loader, model, device)
        outputs = np.array(outputs) >= 0.5
        
        print(f"jaccard score= {jaccard}")
        if accuracy > best_jaccard:
            torch.save(model.state_dict(), MODEL_PATH)
            best_accuracy = accuracy


In [54]:
mod()

OSError: Unable to load weights from pytorch checkpoint file. If you tried to load a PyTorch model from a TF 2.0 checkpoint, please set from_tf=True. 

In [55]:
final_output

''

In [46]:
def post_process(selected):
    return " ".join(set(selected.lower().split()))

result = pd.read_csv(r"C:\Users\shamazha3\Desktop\Text\tweet-sentiment-extraction\sample_submission.csv")
result.loc[:, 'selected_text'] = final_output
result.selected_text = result.selected_text.map(post_process)
result.to_csv(r"C:\Users\shamazha3\Desktop\Text\tweet-sentiment-extraction\sample_submission.csv", index=False)
result.head()

,textID,selected_text
0,f87dea47db,
1,96d74cb729,
2,eee518ae67,
3,01082688c6,
4,33987a8ee5,
